In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
 # Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://realestateproj.s3.ap-southeast-2.amazonaws.com/real-estate-sales.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("real-estate-sales.csv"), header=True, inferSchema=True)

# Show DataFrame
df.show(10)

+-------------+---------+-------------+-------+--------------------+--------------+-----------+-----------+-------------+----------------+------------------+----------------+-----------+--------------------+
|Serial Number|List Year|Date Recorded|   Town|             Address|Assessed Value|Sale Amount|Sales Ratio|Property Type|Residential Type|      Non Use Code|Assessor Remarks|OPM remarks|            Location|
+-------------+---------+-------------+-------+--------------------+--------------+-----------+-----------+-------------+----------------+------------------+----------------+-----------+--------------------+
|      2020348|     2020|   09/13/2021|Ansonia|     230 WAKELEE AVE|      150500.0|   325000.0|      0.463|   Commercial|            NULL|              NULL|            NULL|       NULL|                NULL|
|        20002|     2020|   10/02/2020|Ashford|     390 TURNPIKE RD|      253000.0|   430000.0|     0.5883|  Residential|   Single Family|              NULL|           